## Loading NiBabel for Loading NIFTI Files

In [ ]:
import nibabel as nib
import os
import SimpleITK as sitk
import shutil

## Copying all the labels

In [ ]:
def copy_labels(src, dst):

    for root, dirs, files in os.walk(src):
        
        dest_dir = root.replace(src, dst, 1)
        
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            
        for file in files:
            if file.lower().endswith('.nii.gz'):
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest_dir, file)
                shutil.copy2(src_file, dest_file)
                print(f'Copied: {src_file} to {dest_file}', end="\n\n")

In [ ]:
src = f"[path]"
dest = f"[path]"
copy_labels(src, dest)

### Setting path for printing header information for converted CT image and NIFTI label.

In [ ]:
img = f"[path]"
label = f"[path]"

labelx = f"[path]"

In [ ]:

nifti_image = nib.load(img)
label_image = nib.load(label)

print("=============== Header =================")
print(nifti_image.header)
print("=============== Label =================")
print(label_image.header)

## Using Simple ITK for resampling label

In [ ]:

import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform, zoom

def align_nifti_to_label(nifti_file, label_file, output_file):

    nifti_img = nib.load(nifti_file)
    label_img = nib.load(label_file)

    nifti_data = nifti_img.get_fdata()
    nifti_affine = nifti_img.affine
    nifti_header = nifti_img.header
    
    label_data = label_img.get_fdata()
    label_affine = label_img.affine
    label_header = label_img.header

    nifti_pixdim = nifti_header['pixdim'][1:4]
    label_pixdim = label_header['pixdim'][1:4]

    scale_factors = nifti_pixdim / label_pixdim  
    print(f"Scaling factors: {scale_factors}")
    
    resampled_data = zoom(nifti_data, zoom=scale_factors, order=1) 
    
    new_affine = np.copy(nifti_affine)
    
    new_affine[:3, :3] = label_affine[:3, :3] 
    new_affine[:3, 3] = label_affine[:3, 3]    
    
    
    aligned_nifti_img = nib.Nifti1Image(resampled_data, new_affine, header=label_header)
    
    
    nib.save(aligned_nifti_img, output_file)
    print(f'Aligned NIFTI saved as {output_file}')

nifti_file = f"[path]"  
label_file = f"[path]" 
output_file = f"[path]" 


align_nifti_to_label(nifti_file, label_file, output_file)


In [ ]:

def resample_label_to_image(label_file, image_file, output_file):
    
    print(f"Loading image {image_file} and label {label_file}")
    label_img = sitk.ReadImage(label_file)
    image_img = sitk.ReadImage(image_file)
    
    print("Resampling image label...")

    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(image_img.GetSize())
    resampler.SetOutputSpacing(image_img.GetSpacing())
    resampler.SetOutputOrigin(image_img.GetOrigin())
    resampler.SetOutputDirection(image_img.GetDirection())
    resampler.SetDefaultPixelValue(0)
    resampler.SetTransform(sitk.Transform())  
    resampler.SetInterpolator(sitk.sitkNearestNeighbor) 
    
    resampled_label_img = resampler.Execute(label_img)
    
    sitk.WriteImage(resampled_label_img, output_file)

# resample_label_to_image(label, img, label)


## Executing code for all labels.

In [ ]:
def correct_labels(root_dir):
    
    for root, dirs, files in os.walk(root_dir):
        
        label = None
        image = None
        for file in files:            
            
            if file.lower().endswith('.json'):
                
                continue
            if file.lower().startswith('im') and file.lower().count('_') == 0:
               
               label = os.path.join(root, file)
            elif file.lower().endswith('.nii.gz'):
                
                image = os.path.join(root, file)
        
        if len(files) > 0:
            resample_label_to_image(label, image, label)
            print(f"Label saved to {label}", end="\n\n")

In [ ]:
root_dir = f"[path]"
correct_labels(root_dir)

### Aligning IM0005 Image Labels

In [ ]:
import nibabel as nib
import numpy as np
import vtk

def apply_transform(source, trafo):
    if trafo is None:
        return source
    trafo_filter = vtk.vtkTransformFilter()
    trafo_filter.SetInputConnection(source.GetOutputPort())
    trafo_filter.SetTransform(trafo)
    trafo_filter.Update()
    return trafo_filter.GetOutput()

def nifti_to_vtk_image(nifti_image):
    img_data = nifti_image.get_fdata()
    img_shape = img_data.shape
    img_vtk = vtk.vtkImageData()
    img_vtk.SetDimensions(img_shape)
    img_vtk.AllocateScalars(vtk.VTK_FLOAT, 1)
    
    vtk_array = vtk.util.numpy_support.numpy_to_vtk(img_data.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
    img_vtk.GetPointData().SetScalars(vtk_array)
    
    return img_vtk

def vtk_to_nifti(vtk_image, affine):
    vtk_array = vtk.vtk_to_numpy(vtk_image.GetPointData().GetScalars())
    img_shape = vtk_image.GetDimensions()
    img_data = vtk_array.reshape(img_shape[::-1])

    return nib.Nifti1Image(img_data, affine)

nifti_image = nib.load(img)
label_image = nib.load(label)

vtk_label_image = nifti_to_vtk_image(label_image)

trafo_matrix = nifti_image.affine
trafo = vtk.vtkTransform()
trafo.SetMatrix(trafo_matrix.flatten())

transformed_vtk_image = apply_transform(source=vtk_label_image, trafo=trafo)

transformed_nifti_image = vtk_to_nifti(transformed_vtk_image, nifti_image.affine)

nib.save(transformed_nifti_image, labelx)
